
# 1. Airモジュール  

【概要】  
空気の状態量を保持するためのモジュールである。  
基本構造はCellモジュールと同様のため、cell.ipynbファイルを参考にすること。

In [ ]:
include("./function/vapour.jl")
include("./function/lewis_relation.jl")

1.5737642053776505e-7

## 2. 構造体：Air

コンストラクタ(データの型)：
- name：名称(String)
- vol：体積(Float64)
- temp：温度(Float64)
- rh：相対湿度(Float64)

In [ ]:
Base.@kwdef mutable struct Air
    num::Int       = 0     #= 室番号 =#
    name::String   = "NoName"  #= 名称  =#
    dx::Float64    = 0.0   #= 幅x   =#
    dy::Float64    = 0.0   #= 高さy =#
    dz::Float64    = 0.0   #= 奥行z =#
    vol::Float64   = 0.0   #= 体積  =#
    temp::Float64  = 0.0   #= 温度  =#
    rh::Float64    = 0.0   #= 相対湿度  =#
    pv::Float64    = 0.0   #= 水蒸気圧  =#
    ah::Float64    = 0.0   #= 絶対湿度  =#
    p_atm::Float64 = 101325.0 #= 空気の全圧（≒大気圧）=# 
    CO2::Float64   = 0.0   #= CO2濃度  =#
    H_in::Float64  = 0.0      #= 発熱源による発熱量 =#
    H_wall::Float64 = 0.0     #= 壁体からの熱流量 =#
    H_vent::Float64 = 0.0     #= 換気による熱流量 =#
    J_in::Float64   = 0.0     #= 発熱源による発湿量 =#
    J_wall::Float64 = 0.0     #= 壁体からの水分流量 =#
    J_vent::Float64 = 0.0     #= 換気による水分流量 =#
end 

Air


## 2.2 Air構造体を用いた状態量・物性値を取得する関数：
#### 構造体パラメーター
- name(Air) : （対象とする空間などの）名称
- vol(Air)  : 空間の体積

#### 熱水分状態量
- temp(Cell): 温度
- rh(Cell)  : 相対湿度
- miu(Cell) : 水分化学ポテンシャル（温度・相対湿度から換算）
- pv(Cell)  : 水蒸気圧（温度・相対湿度から換算）

In [ ]:
# 構造体パラメーター
num(state::Air)= state.num
name(state::Air)= state.name
vol(state::Air)= state.vol
material_name(state::Air)= typeof(state)

# 熱水分状態量
temp(state::Air)= state.temp
rh(state::Air) = state.rh
miu(state::Air) = convertRH2Miu( temp = state.temp, rh = state.rh )
pv(state::Air)  = convertRH2Pv( temp = state.temp, rh = state.rh )
ah(state::Air)  = convertPv2AH( patm = state.p_atm, pv = pv(state) )
p_atm(state::Air)= state.p_atm

# 流入熱量
H_in(state::Air)   = state.H_in
H_wall(state::Air) = state.H_wall
H_vent(state::Air) = state.H_vent
J_in(state::Air)   = state.J_in
J_wall(state::Air) = state.J_wall
J_vent(state::Air) = state.J_vent

########################################
# 入力用の関数
function set_num(air::Air, num::Int) 
    air.num    = num end
function set_name(air::Air, num::String) 
    air.name   = name end
function set_temp(air::Air, temp::Float64) 
    air.temp   = temp  end
function set_rh(air::Air, rh::Float64) 
    air.rh     = rh    end
function set_pv(air::Air, pv::Float64) 
    air.pv     = pv    end
function set_ah(air::Air, ah::Float64) 
    air.ah     = ah    end
function set_p_atm(air::Air, p_atm::Float64) 
    air.p_atm  = p_atm end
function set_H_in(air::Air, H_in::Float64) 
    air.H_in   = H_in end    
function set_H_wall(air::Air, H_wall::Float64) 
    air.H_wall = H_wall end    
function set_H_vent(air::Air, H_vent::Float64) 
    air.H_vent = H_vent end    
function set_J_in(air::Air, J_in::Float64) 
    air.J_in   = J_in end    
function set_J_wall(air::Air, J_wall::Float64) 
    air.J_wall = J_wall end    
function set_J_vent(air::Air, J_vent::Float64) 
    air.J_vent = J_vent end            

########################################
# 積算用の関数
function add_H_in(air::Air, H_in::Float64) 
    air.H_in   = air.H_in + H_in end    
function add_H_wall(air::Air, H_wall::Float64) 
    air.H_wall = air.H_wall + H_wall end    
function add_H_vent(air::Air, H_vent::Float64) 
    air.H_vent = air.H_vent + H_vent end    
function add_J_in(air::Air, J_in::Float64) 
    air.J_in   = air.J_in + J_in end    
function add_J_wall(air::Air, J_wall::Float64) 
    air.J_wall = air.J_wall + J_wall end    
function add_J_vent(air::Air, J_vent::Float64) 
    air.J_vent = air.J_vent + J_vent end            

J_vent (generic function with 1 method)

## 3. 熱水分収支式

### 3.1 一般解

室内における収支式は、一般に以下のように表される。  

熱収支式（１）：
$$c_a \rho_a V \frac{\partial T_i}{\partial t} = -\Sigma_{k=1}^{m}S_k \alpha(T_{i} - T_{S_{k}}) -\Sigma_{j=1}^{n} ( Q_{ij} c_a \rho_a T_i - Q_{ji} c_a \rho_a T_j ) + H_i$$

水分収支式（２）：
$$ \rho_a V \frac{\partial X_i }{\partial t} = -\Sigma_{k=1}^{m}S_k \alpha^{'}(X_{i} - X_{S_{k}}) -\Sigma_{j=1}^{n} ( Q_{ij} \rho_a X_i - Q_{ji} \rho_a X_j ) + H_i^{'}$$

これらを単純化すると以下のように記述することができる。

熱収支式（１'）：
$$c_a \rho_a V \frac{\partial T}{\partial t} = H_w + H_v + H_i$$

水分収支式（２'）：
$$\rho_a V \frac{\partial X_i }{\partial t}  = J_w + J_v + H_i^{'}$$  

ここで、  
- $H_w, J_w$：壁体との熱・湿気交換  
- $H_v, J_v$：換気による熱・湿気交換  
- $H_i, J_i^{'}$：発熱量・発湿量

#### 熱収支式（１'）：

In [4]:
# 基礎式
function cal_energy_balance(temp::Float64, ah::Float64, vol::Float64, Hw::Float64, Hv::Float64, Hi::Float64, dt::Float64)
    ca = 1005.0 + 1846.0 * ah # 乾き空気＋水蒸気の比熱
    rho=  353.25 / temp # ボイルシャルルの法則より
    return temp + ( Hw + Hv + Hi ) / ( ca * rho * vol / dt )   
end

# air構造体を用いた式
cal_energy_balance(air::Air, dt::Float64) = cal_energy_balance(temp(air), ah(air), vol(air), H_wall(air), H_vent(air), H_in(air), dt)
cal_newtemp(air::Air, dt::Float64) = cal_energy_balance(air, dt)

cal_newtemp (generic function with 1 method)

熱収支式（１）の形を用いると以下のような式が考えられる。

In [10]:
# 2024/1/30加筆中
#function cal_energy_balance(air::Air, wall::Array{Wall, 1}, dt::Float64) 
#    cal_energy_balance(temp(air), ah(air), vol(air), H_wall(air), H_vent(air), H_in(air), dt)
#end

#### 水分収支式（２'）：

In [11]:
# 基礎式
function cal_moisture_balance( temp::Float64, ah::Float64, vol::Float64, Jw::Float64, Jv::Float64, Ji::Float64, dt::Float64)
    rho=  353.25 / temp
    return ah + ( Jw + Jv + Ji ) / ( rho * vol / dt )   
end

# air構造体を用いた式
cal_moisture_balance(air::Air, dt::Float64) = cal_moisture_balance( temp(air), ah(air), vol(air), J_wall(air), J_vent(air), J_in(air), dt)
cal_newRH(air::Air, dt::Float64) = convertAH2RH( temp = temp(air), patm = p_atm(air), ah = cal_moisture_balance(air, dt))

cal_newRH (generic function with 1 method)

In [12]:
test_air = Air(temp = 293.15, rh = 0.6, vol = 1.0, J_wall = 0.001)

Air("NoName", 0.0, 0.0, 0.0, 1.0, 293.15, 0.6, 101325.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.0)

In [13]:
cal_newRH(test_air, 1.0)

0.6561378416360885